In [ ]:
### PREREQUIRMENTS
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [ ]:
study_arm_subject = pd.read_csv('tables/study_arm_subject.csv') # duplicates (probably bc of different AGE_EVENTS)
lab_tests = pd.read_csv('tables/lab_tests.csv')

In [ ]:
### manually assigned by oreferred names -> macht kein sinn weil da extrem viele nans sind-..
#### ASSIGNEMENT BY PANEL_NAMES

def is_test_per_panel(panel_names, df):
    return {entry: df['PANEL_NAME'].value_counts().get(entry, 0) for entry in panel_names}
### cell types
cell_type_PANEL_NAMEs = [
    'Blood Cell Count: ',
    'Blood Cell Count with Differential: ',
    'Blood cell counts: ',
    'CBC: ',
    'CBC/Differential Results: ',
    'CBC_DIFFERENTIAL: ',
    'CBC_Results: ',
    'Complete Blood Cell Count: ',
    'Complete blood count: ',
    'DIFFERENTIAL: ',
    'HEMATOLOGY: ',
    'Hematology: ',
    'Hematology test: ',
]

cell_types_PANEL_NAMEs_trimmed = [x[:-2] for x in cell_type_PANEL_NAMEs] ## trim the bins names as i added glutter to have it nicer... 
cell_types_all_tests = lab_tests[lab_tests['PANEL_NAME'].isin(cell_types_PANEL_NAMEs_trimmed)]
print(is_test_per_panel(cell_types_PANEL_NAMEs_trimmed, lab_tests))


### biomarkers
biomarker_PANEL_NAMEs = [
    'Blood Flow Cytometry: ',
    'Protein or Enzyme Type Measurement: ',
    ]
biomarker_PANEL_NAMEs_trimmed = [x[:-2] for x in biomarker_PANEL_NAMEs] ## trim the bins names as i added glutter to have it nicer... 
biomarker_all_tests = lab_tests[lab_tests['PANEL_NAME'].isin(biomarker_PANEL_NAMEs_trimmed)]
print(is_test_per_panel(biomarker_PANEL_NAMEs_trimmed, lab_tests))
biomarker_all_studies_array = biomarker_all_tests['STUDY_ACCESSION'].unique()
biomarker_all_studies = study_arm_subject[study_arm_subject['STUDY_ACCESSION'].isin(biomarker_all_studies_array)]


# ### Fasting
fasting_PANEL_NAMEs = [
    'FASTING_BLOOD_CHEMISTRY: ',
    'Fasting Lipid Panel: ',
    'Fasting Lipid Profile: ',
    'Fasting Lipids: '
]
fasting_PANEL_NAMEs_trimmed = [x[:-2] for x in fasting_PANEL_NAMEs] ## trim the bins names as i added glutter to have it nicer... 
fasting_all_tests = lab_tests[lab_tests['PANEL_NAME'].isin(fasting_PANEL_NAMEs_trimmed)]
print(is_test_per_panel(fasting_PANEL_NAMEs_trimmed, lab_tests))


# ### antigens
antigens_PANEL_NAMEs = [
    'Basophil: ',
    'Immunology Test: ',
    'Serum Protein Electrophoresis: '
]
antigens_PANEL_NAMEs_trimmed = [x[:-2] for x in antigens_PANEL_NAMEs] ## trim the bins names as i added glutter to have it nicer... 
antigens_all_tests = lab_tests[lab_tests['PANEL_NAME'].isin(antigens_PANEL_NAMEs_trimmed)]
print(is_test_per_panel(antigens_PANEL_NAMEs_trimmed, lab_tests))

# ### chemistry
chemistry_PANEL_NAMEs=[
    'BASIC_CHEMISTRY: ',
    'BLOOD_CHEMISTRY: ',
    'Blood Chemistry: ',
    'Blood Chemistry Measurement: ',
    'Blood Chemistry Test: ',
    'Chemistry Test: ',
    'Clinical Chemistry: ',
    'Comprehensive Metabolic Panel: ',
    'Hematology: ',
    'Serum Chemistry: '
]
chemistry_PANEL_NAMEs_trimmed = [x[:-2] for x in chemistry_PANEL_NAMEs] ## trim the bins names as i added glutter to have it nicer... 
chemistry_all_tests = lab_tests[lab_tests['PANEL_NAME'].isin(chemistry_PANEL_NAMEs_trimmed)]
print(is_test_per_panel(chemistry_PANEL_NAMEs_trimmed, lab_tests))


# ### creatinin
# Creatinine test: 
# Protein or Enzyme Type Measurement: 

# SERUM_CREATININE: 


# ### other

# ### urea
# Urinalysis: 
# Urine

# ### liver
# LIVER_PANEL: 


In [ ]:
def create_directory(directory_path):
    try:
        os.makedirs(directory_path, exist_ok=True)
        print(f"Directory '{directory_path}' created successfully.")
    except OSError as error:
        print(f"Creation of the directory '{directory_path}' failed. Error: {error}")


def violinplot_overall(df, x_topic='GENDER', y_topic='MAX_SUBJECT_AGE_IN_YEARS', hue_split='GENDER', bin_name='test_bin'):
    sns.violinplot(x=x_topic, y=y_topic, hue=hue_split, data=df[[x_topic, y_topic]], split=True)
    plt.savefig(f'{bin_name}/violin_overall.png')
    

def violinplot_per_criteria(df, x_topic='STUDY_ACCESSION', y_topic='MAX_SUBJECT_AGE_IN_YEARS', hue_split='GENDER', bin_name='test_bin'):
    plt.figure(figsize=(len(df[x_topic].unique()) * 1.5, 6))
    sns.violinplot(x=x_topic, y=y_topic, hue=hue_split, data=df[[x_topic, hue_split, y_topic]], split=True, cut=0.5, inner=None)
    plt.xticks(rotation=45)
    plt.axhline(y=0)
    plt.tick_params(axis='y', bottom=True)
    plt.savefig(f'{bin_name}/violin_per_{x_topic}.png')


def create_bin_report(tests_df, bin_name='TEST'):
    create_directory(bin_name)
    to_write_array = []
    studies_unique = tests_df['STUDY_ACCESSION'].unique()
    print(f'Studies for {bin_name}: {len(studies_unique)}')
    to_write_array.append(f'Amount studies: {len(studies_unique)}')
    
    all_studies = study_arm_subject[study_arm_subject['STUDY_ACCESSION'].isin(studies_unique)]
    bin_study_arm_subject = study_arm_subject[study_arm_subject['STUDY_ACCESSION'].isin(all_studies['STUDY_ACCESSION'])]
    bin_study_arm_subject_no_douplicates = bin_study_arm_subject.drop_duplicates('SUBJECT_ACCESSION')
    print(f'Amount of Subjects: {bin_study_arm_subject.shape[0]}')
    to_write_array.append(f'Amount subjects: {bin_study_arm_subject.shape[0]}')
    print(f'Amount of Subjects no duplicates: {bin_study_arm_subject_no_douplicates.shape[0]}')
    to_write_array.append(f'Amount subjects no douplicates: {bin_study_arm_subject_no_douplicates.shape[0]}')
    with open(f'{bin_name}/report.txt', 'w') as file:
        for item in to_write_array:
            file.write(f"{item}\n")
        
    # here i´d like to create a bunch of violinplots in a folder
    violinplot_overall(bin_study_arm_subject_no_douplicates, 'GENDER', 'MAX_SUBJECT_AGE_IN_YEARS', 'GENDER', bin_name)
    violinplot_per_criteria(bin_study_arm_subject_no_douplicates,  'STUDY_ACCESSION', 'MAX_SUBJECT_AGE_IN_YEARS', 'GENDER', bin_name)

    print(f'Age event: {len(bin_study_arm_subject_no_douplicates["AGE_EVENT"].unique())}')
    to_write_array.append(f'Age events: {len(bin_study_arm_subject_no_douplicates["AGE_EVENT"].unique())}')
    violinplot_per_criteria(bin_study_arm_subject_no_douplicates,  'AGE_EVENT', 'MAX_SUBJECT_AGE_IN_YEARS', 'GENDER', bin_name)
    return bin_study_arm_subject_no_douplicates


        

In [ ]:
biomarker_study_arm_subject = create_bin_report(biomarker_all_tests, 'Biomarker')
biomarker_study_arm_subject.head()

In [ ]:
antigen_study_arm_subject = create_bin_report(antigens_all_tests, 'Antigens')
antigen_study_arm_subject.head()

In [ ]:
chemistry_study_arm_subject = create_bin_report(chemistry_all_tests, 'Chemistry')
chemistry_study_arm_subject.head()

In [ ]:
fasting_study_arm_subject = create_bin_report(fasting_all_tests, 'Fasting')


In [ ]:
cell_types_study_arm_subject = create_bin_report(cell_types_all_tests, 'CellTypes')
cell_types_study_arm_subject.head()